<a href="https://colab.research.google.com/github/thiagorainmaker77/liar_dataset/blob/master/analise_sentimento_ok.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [28]:
!pip install langdetect

In [74]:
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')
import nltk
from nltk.sentiment.vader import SentimentIntensityAnalyzer
nltk.download('vader_lexicon')
import string
import nltk
from nltk import PorterStemmer
import re
nltk.download('stopwords')
nltk.download('wordnet')
stopwords = nltk.corpus.stopwords.words('english')
from gensim import corpora
import gensim
from sklearn.metrics import confusion_matrix, f1_score, classification_report
from sklearn.naive_bayes import MultinomialNB
from sklearn.pipeline import Pipeline
from sklearn.feature_extraction.text import CountVectorizer



[nltk_data] Downloading package vader_lexicon to /root/nltk_data...
[nltk_data]   Package vader_lexicon is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [0]:
uri_train  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/train.tsv'
uri_valid  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/valid.tsv'
uri_test  = 'https://raw.githubusercontent.com/thiagorainmaker77/liar_dataset/master/test.tsv'

    
df_train = pd.read_table(uri_train,
                             names = ['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])

    
df_valid = pd.read_table(uri_valid,
                             names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue'])


df_test = pd.read_csv(uri_test, sep='\t', 
                            names =['id',	'label'	,'statement',	'subject',	'speaker', 	'job', 	'state',	'party',	'barely_true_c',	'false_c',	'half_true_c',	'mostly_true_c',	'pants_on_fire_c',	'venue']) 


df = pd.concat([df_train, df_valid])

In [0]:
liar_list = [df,df_test]

n = 10

subject = liar_list[0].groupby(['subject']).count().label.sort_values(ascending=False)[:n].index.tolist()
speaker = liar_list[0].groupby(['speaker']).count().label.sort_values(ascending=False)[:n].index.tolist()
job = liar_list[0].groupby(['job']).count().label.sort_values(ascending=False)[:n].index.tolist()
state = liar_list[0].groupby(['state']).count().label.sort_values(ascending=False)[:n].index.tolist()
party = liar_list[0].groupby(['party']).count().label.sort_values(ascending=False)[:n].index.tolist()
venue = liar_list[0].groupby(['venue']).count().label.sort_values(ascending=False)[:n].index.tolist()


In [0]:
class MultiColumnLabelEncoder:
  
  
    countV = CountVectorizer(analyzer='word', binary=True, decode_error='strict', encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)
    
    def __init__(self,columns = None):
        self.columns = columns # array of column names to encode

    def fit(self,X,y=None):
        return self # not relevant here

    def transform(self,X):
        '''
        Transforms columns of X specified in self.columns using
        LabelEncoder(). If no columns specified, transforms all
        columns in X.
        '''
        output = X.copy()
        if self.columns is not None:
            for col in self.columns:
                output[col] = countV().fit_transform(output[col])
        else:
            for colname,col in output.iteritems():
                output[colname] = countV().fit_transform(col)
        return output

    def fit_transform(self,X,y=None):
        return self.fit(X,y).transform(X)

In [0]:
features_of_interest = state+subject+speaker+party+venue+job
to_encode = [('speaker',speaker),
            ('party',party),
            ('subject',subject),
            ('state',state),
            ('venue',venue),
            ('job',job)]
for i in liar_list:
    for var in to_encode:
        for value in var[1]:
            i[value] = i[var[0]].apply(lambda x: x==value)

In [0]:
truth = {'false':0.,'half-true':0.5,'mostly-true':0.75,'true':1.,'pants-fire':-0.25,'barely-true':0.25} 


for i in liar_list:
    i['numer_truth'] = i['label'].apply(lambda x: truth[x])

In [0]:
def score(file):
  data_pre = pd.read_csv(file, sep='\n', names = ["label"]) 
  score = f1_score(df_test['label'],data_pre, average='micro')
  return score

In [0]:

sid = SentimentIntensityAnalyzer()

ps = PorterStemmer()
wn = nltk.WordNetLemmatizer()

def clean_stem (sent, seq=False):
    temp1=sent
    temp2 = re.split('\W+',temp1)
    temp3 = [ps.stem(x) for x in temp2 if x not in stopwords]
    return temp3
def sentiment(x):
    score = sid.polarity_scores(x)
    return score['compound']

In [0]:
for i in liar_list:
    i['sentiment'] = i['statement'].apply(lambda x: sentiment(' '.join(clean_stem(x))))

In [0]:
df_test['sentiment'] = df_test['statement'].apply(lambda x: sentiment(' '.join(clean_stem(x))))

In [0]:
countV = CountVectorizer(analyzer='word', binary=True, decode_error='strict', encoding='utf-8', input='content',
        lowercase=True, max_df=0.5, max_features=None, min_df=1,
        ngram_range=(1, 1), preprocessor=None, stop_words='english',
        strip_accents=None, token_pattern='(?u)\\b\\w\\w+\\b',
        tokenizer=None, vocabulary=None)

In [92]:
countV.fit_transform(df['statement'].values)



<11524x12590 sparse matrix of type '<class 'numpy.int64'>'
	with 114275 stored elements in Compressed Sparse Row format>

In [0]:

nb_cv = Pipeline([
        ('NBCV', countV),
        ('nb_clf', MultinomialNB(alpha=1.5, class_prior=None, fit_prior=True))])

In [91]:
nb_cv.fit(df['statement'] + df['sentiment'],df['label'])





TypeError: ignored

In [0]:
arquivo = 'best_result.txt'

vf = open(file, 'w+')
for index, row in df_test.iterrows():
  text = {row['statement'].add(row['sentiment'])}
  pred_a = class_a.predict(text)
  prend = pred_a[0]   
  vf.write(str(prend)+'\n')

In [80]:
print(score(arquivo))

0.2628255722178374


In [81]:
countV.fit_transform(df['statement'].values)

<11524x12590 sparse matrix of type '<class 'numpy.int64'>'
	with 114275 stored elements in Compressed Sparse Row format>

In [0]:
nb_cv